In [3]:
import pandas as pd
import glob
import xml.etree.ElementTree as ET
import pickle
import os
import numpy as np
from tqdm import tqdm


In [4]:
with open('/iris/u/nathu/temporal-LMs/learned_updating/data/NYT_aggregated_info.pkl', 'rb') as f:
    NYT_aggregated_info = pickle.load(f)

In [5]:
#drop rows with no text

#make sure we were able to correctly parse the date
NYT_aggregated_info['year'] = NYT_aggregated_info['date'].apply(lambda x: None if x.split('-')[0] == 'None' else int(x.split('-')[0]))
NYT_aggregated_info = NYT_aggregated_info.dropna(subset=['year', 'text', 'title'])
NYT_aggregated_info = NYT_aggregated_info[NYT_aggregated_info['year'] <= 2008]
NYT_aggregated_info['date'] = pd.to_datetime(NYT_aggregated_info['date'])
NYT_aggregated_info.sort_values(by=['date'], inplace=True)
NYT_aggregated_info['merged_text'] = NYT_aggregated_info['text'].apply(lambda x:  '\n'.join(x))

In [6]:
print(len(NYT_aggregated_info))
#filter out paid notices/obituatries
NYT_aggregated_info = NYT_aggregated_info[~NYT_aggregated_info['title'].apply(lambda x: x[:12] == 'Paid Notice:')]
print('filtered paid_notices', len(NYT_aggregated_info))
#earnings reports
NYT_aggregated_info = NYT_aggregated_info[~NYT_aggregated_info.apply(lambda x: '*** COMPANY REPORTS **' in x['merged_text'], axis = 1)]
print('filtered earnings reports', len(NYT_aggregated_info))
NYT_aggregated_info = NYT_aggregated_info[~NYT_aggregated_info.apply(lambda x: 'To the Editor:' in x['merged_text'], axis = 1)]
print('filtered "to the editor:"', len(NYT_aggregated_info))
NYT_aggregated_info = NYT_aggregated_info[NYT_aggregated_info.apply(lambda x: len(x['merged_text']) > 500, axis = 1)]
print('filtered for 500+ char', len(NYT_aggregated_info))

1829595
filtered paid_notices 1698297
filtered earnings reports 1595048
filtered "to the editor:" 1481145
filtered for 500+ char 1365292


In [7]:
qa_pretrain_split = NYT_aggregated_info[NYT_aggregated_info['date'] <= pd.to_datetime('1988-09-30')]
qa_pretrain_val_split = NYT_aggregated_info[(NYT_aggregated_info['date'] >= pd.to_datetime('1988-10-01'))
                                            &(NYT_aggregated_info['date'] <= pd.to_datetime('1988-12-31'))]
meta_train_split = NYT_aggregated_info[(NYT_aggregated_info['date'] > pd.to_datetime('1989-01-01')) 
                                       & (NYT_aggregated_info['date'] <= pd.to_datetime('1992-12-31'))]
eval_split = NYT_aggregated_info[(NYT_aggregated_info['date'] > pd.to_datetime('1993-01-01')) 
                                 & (NYT_aggregated_info['date'] <= pd.to_datetime('1994-12-31'))]
print('qa_pretrain_split', len(qa_pretrain_split))
print('qa_pretrain_val_split', len(qa_pretrain_val_split))
print('meta_train_split', len(meta_train_split))
print('eval_split', len(eval_split))




qa_pretrain_split 141595
qa_pretrain_val_split 20479
meta_train_split 283947
eval_split 114896


In [8]:
qa_pretrain_split.to_json('../data/nyt/qa_pretrain_split.json')
qa_pretrain_val_split.to_json('../data/nyt/qa_pretrain_val_split.json')
meta_train_split.to_json('../data/nyt/meta_train_split.json')
eval_split.to_json('../data/nyt/eval_split.json')